# 클라우드 환경에서 Darnet을 이용한 YOLOv4 객체 탐지기 만들기 (with 무료 GPU)


Reference : https://pjreddie.com/darknet/yolo/#demo

(배경지식) 리눅스 커맨드 모음. 구글링을 활용해 사용법을 익히자

%cd [directory]
 -  해당 디렉토리로 이동하는 명령어


!ls [option]
 -  현 directory의 파일과 디렉토리를 나열


!rm
 - 파일, 디렉토리 삭제


!mkdir
 - 새로운 디렉토리 생성


!mv /위치/*.txt /위치
 - 파일 이동 / 파일 이름 변경


!pwd
 - 현 위치를 알려줌

!sed
 - 파일 편집기

!zip -r (복사위치).zip (복사할 대상 경로)
 - 복사할 대상 경로의 파일,폴터들을 복사위치에 zip시킴

!unzip
 - 압축풀기

참고 : https://jgefroh.medium.com/a-beginners-guide-to-linux-command-line-56a8004e2471

참고 : 
https://devkingdom.tistory.com/154

가장 중요한 것은 directory 위치!! 언제든 현재 디렉토리를 모를 땐  ***!pwd*** 를 실행

# 1. Pretrained Model로 객체탐지 해보기
수정-노트설정-GPU활성화
런타임이 처음이 아니라면 초기화

다크넷 다운로드 및 실행파일 만들기

* CUDA? CUDA(Computed Unified Device Architecture)는 NVIDIA에서 개발한 GPU 개발 툴
 => GPU 병렬 프로그래밍

In [ ]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
%cd darknet

In [ ]:
# change makefile to have GPU and OPENCV enabled
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version
# make darknet (build) 몇분 걸림
!make

유용한 함수 만들기(이미지 보여주기, 업로드 ,다운로드)

In [ ]:
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

def download(path):
  from google.colab import files
  files.download(path)

Pretrained Model 다운 받기

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

#!wget https://pjreddie.com/media/files/yolov3.weights     ----------->v3

(선택1) 샘플 이미지로 해보기

In [ ]:
!./ detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights data/dog.jpg -thresh 0.5 -ext_output -dont_show
darknetimShow('predictions.jpg')

(선택2) 로컬 파일을 업로드 해서 적용하기 및 결과 다운받기

In [ ]:
upload()
파일이름 = input("적용할 파일의 이름 입력: ")
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights "$파일이름"* 
imShow('predictions.jpg')

In [ ]:
download('predictions.jpg')

(선택3) 비디오에 적용하기

In [ ]:
upload()
파일이름 = input("적용할 파일의 이름 입력: ")
!./darknet detector demo cfg/coco.data cfg/yolov4.cfg yolov4.weights -dont_show "$파일이름"* -i 0 -out_filename results.avi

In [ ]:
download('results.avi')

(선택4) 구글 드라이브 내의 파일로 적용하기

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#사진파일
파일경로 = input("적용할 파일의 경로 입력: ")
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights "$파일경로"*
imShow('predictions.jpg')

In [ ]:
#동영상파일
파일경로 = input("적용할 파일의 경로 입력: ")
!./darknet detector demo cfg/coco.data cfg/yolov4.cfg yolov4.weights -dont_show "$파일경로"* -i 0 -out_filename results.avi
download('results.avi')

---

# 2. Custom Data로 나의 모델 만들기 
(런타임 - 런타임 초기화)

**준비물**: 

**1.** obj.data

**2.** obj.names

**3.** yolov3_custom.cfg

**4.** 라벨링된 사진의 zip폴더

**5.** 1,2,3,4 가 들어있는 폴더


**학습으로 얻는 것:**

yolov3_custom_last.weight 파일

In [ ]:
!git clone https://github.com/AlexeyAB/darknet
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!/usr/local/cuda/bin/nvcc --version
!make

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

준비물 폴더의 이름을 symbol화 하는 과정을 통해 귀찮은 과정 극복 + 일반화

In [ ]:
path = input("준비물 폴더 경로 입력")
!ln -s "$path"* /mydrive
# 예시 :  /content/gdrive/Shareddrives/PHASOR/2팀/YOLOv4_ObjDetect

In [ ]:
!cp /mydrive/images.zip ../
#!cp /mydrive/test.zip ../
!unzip ../images.zip -d data/obj

In [ ]:
!cp /mydrive/yolov4-obj.cfg ./cfg
!cp /mydrive/obj.names ./data
!cp /mydrive/obj.data  ./data

In [ ]:
#일부 파일에 문제가 있어서 삭제하지 않으면 학습에 오류가 납니다. 실행해주세요
%cd data/obj/
!rm image903.jpg
!rm image903.txt
!rm image1086.jpg
!rm image1086.txt
%cd ..
%cd ..
!ls data/obj/
#1086,903

In [ ]:
# data 폴더에 train.txt를 만드는데, 그 파일안에는 이미지 파일의 경로가 써있음
import os

image_files = []
os.chdir(os.path.join("data", "obj"))
for filename in os.listdir(os.getcwd()):
    if filename.endswith(".jpg"):
        image_files.append("data/obj/" + filename)
os.chdir("..")
with open("train.txt", "w") as outfile:
    for image in image_files:
        outfile.write(image)
        outfile.write("\n")
    outfile.close()
os.chdir("..")

In [ ]:
#import os

#image_files = []
#os.chdir(os.path.join("data", "test"))
#for filename in os.listdir(os.getcwd()):
#    if filename.endswith(".jpg"):
#        image_files.append("data/test/" + filename)
#os.chdir("..")
#with open("test.txt", "w") as outfile:
#    for image in image_files:
#        outfile.write(image)
#        outfile.write("\n")
#    outfile.close()
#os.chdir("..")

**학습하기!**

구글 코랩은 일정시간 동작이 없으면 런타임 연결을 끊음.
따라서 다음의 코드를 F12를 눌러 콘솔창에 입력후 엔터를 눌러 10분에 한번씩 코드 창을 누르는 동작을 하도록 만들어 런타임 연결이 끊기지 않도록 하게 하자.


function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button").click() 
}setInterval(ClickConnect, 600000)

(선택1) 처음부터 학습하려면

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137
# !wget http://pjreddie.com/media/files/darknet53.conv.74  ------->v3

In [ ]:
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg yolov4.conv.137 -dont_show
# !./darknet detector train data/obj.data cfg/yolov3_custom.cfg darknet53.conv.74 -dont_show    ------->v3

(선택2) 이어 학습하고 싶다면 아래를 실행

In [ ]:
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg /mydrive/backup/yolov4-obj_last.weights -dont_show

accuracy 보고싶다면

In [ ]:
imShow('chart.png')

최적의 weight를 찾을 수 있는 평가 지표를 출력해줌. 단, test이미지 파일이 존재해야함. 없다면 실행불가

In [ ]:
!./darknet detector map data/obj.data cfg/yolov4-obj.cfg /mydrive/backup/yolov4-obj_1000.weights

---

# 3. Custom Model로 객체탐지 해보기
수정-노트설정-GPU활성화

런타임 초기화

In [ ]:
!git clone https://github.com/AlexeyAB/darknet
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!/usr/local/cuda/bin/nvcc --version
!make

def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline
  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)
  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)
def download(path):
  from google.colab import files
  files.download(path)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
path = input("준비물 폴더 경로 입력")
!ln -s "$path"* /mydrive
# 예시 :  /content/gdrive/Shareddrives/PHASOR/2팀/yolov4_tutorial

In [ ]:
!cp /mydrive/yolov4-obj.cfg ./cfg
!cp /mydrive/obj.names ./data
!cp /mydrive/obj.data  ./data

In [ ]:
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov4-obj.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov4-obj.cfg
%cd ..

이미지

In [ ]:
path = input("테스트 사진 경로 입력")
!./darknet detector test data/obj.data cfg/yolov4-obj.cfg /mydrive/backup/yolov4-obj_last.weights "$path"* -thresh 0.3 -dont show
imShow('predictions.jpg')
# 예시 : /mydrive/test_images/pothole.jpg

비디오

In [ ]:
upload()
비디오파일이름 = input("테스트 영상 이름 입력")

In [ ]:
!./darknet detector demo data/obj.data cfg/yolov4-obj.cfg /mydrive/backup/yolov4-obj_last.weights -dont_show -thresh 0.5 "$비디오파일이름"* -i 0 -out_filename results.avi
download('results.avi')